In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import evaluate
from utils import save_response

# PHI3 SHORT STORY MODEL 

In [2]:
tokenizer_model = "unsloth/Phi-3-mini-4k-instruct"
model_id = "oztrkoguz/phi3_short_story_merged_bfloat16"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
model = AutoModelForCausalLM.from_pretrained(model_id).to("cuda")

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

In [3]:
input_text = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 2 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint

### Response:
"""
print(input_text)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 2 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint

### Response:



In [4]:

hyperparams = {
    "max_new_tokens": 512, 
    "temperature":0.6, 
    "repetition_penalty":2.0, 
    "no_repeat_ngram_size":1
}
encoded_input = tokenizer([input_text], return_tensors='pt').to("cuda")
streamer = TextStreamer(tokenizer)
response = model.generate(input_ids=encoded_input.input_ids, 
                        streamer=streamer, 
                        max_new_tokens=hyperparams["max_new_tokens"], 
                        temperature=hyperparams["temperature"], 
                        repetition_penalty=hyperparams["repetition_penalty"], 
                        no_repeat_ngram_size=hyperparams["no_repeat_ngram_size"]).to("cuda")

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 2 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint

### Response:


c:\Users\pmahi\OneDrive\Desktop\MP\Drexel\Courses\CS614-900 - Applications in Machine Learning\Projects\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\pmahi\OneDrive\Desktop\MP\Drexel\Courses\CS614-900 - Applications in Machine Learning\Projects\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:674: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


It's nighttime at Times Square as people run away from horrifying creatures known only under one name - ZOMBIES! The scene shifts between running through crowded streets of NYC while dodging dead bodies left behind after being attacked or killed; then suddenly stopping when they see something ahead (a car) which turns out not really dangerous but just another obstacle along their path towards safety... Then we cut back into our main character walking down Broadway before he starts joggling his way up West Street until finally reaching Central Park near Columbus Circle without any problems whatsoever except maybe some minor injuries caused due too much stress over these past few minutes.. He continues forward now faster than ever because there aren´t many places around here anymore so if someone wants escape this hellhole better hurry!!<|endoftext|>


In [5]:
output = tokenizer.decode(response[0], skip_special_tokens=True)
print(output)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Input:
Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 2 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint

### Response:
It's nighttime at Times Square as people run away from horrifying creatures known only under one name - ZOMBIES! The scene s

In [6]:
save_response("PHI3-SHORT-STORY", hyperparams, output)

In [7]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [8]:
# print(round(results["mean_perplexity"], 2))